In [2]:
#extract information from each job posting and append it to a list 

from bs4 import BeautifulSoup
from helium import *
import time 
from datetime import date
import pandas as pd

def scrape():
    
    url = 'https://www.usajobs.gov/?c=opportunities'
    driver = start_chrome(url, headless = True)
    driver.get(url)
                   
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    hrefs_list = []
    jobs_list = []
    id_scrape_list = []
    name_scrape_list = []
    agency_scrape_list = []
    
    
# get all hraf link for each job category
    for link in soup.find_all("a",{"class": "usajobs-landing-find-opportunities__job"}):
        hrefs_list.append(link['href'])
        
#open each category link in new browser
    for term in hrefs_list:
        new_url = 'https://www.usajobs.gov'+ term
        driver = start_chrome(new_url, headless = True)
        driver.get(new_url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
#scrape each link for all job box        
        job_link = soup.find_all("a",{'class':'usajobs-search-result--card__body'})
        
#scrape each box for text information about the job (department, agency, and summary) and append to scrape list              
        for job in job_link:
        
            #Get each document-id
            data = [item.get("data-document-id") for item in soup.find_all("a")]
            #Remove "None"
            dataID = list(filter(None, data))
            #Remove duplicates
            no_dups = list(dict.fromkeys(dataID))
            #Append to final list
            id_scrape_list.append((no_dups))
            
            #Get each job name
            #title = [item.get("itemprop") for item in soup.find_all("h3")]
            #title1 = list(filter(None, title))
            
            name_box = job.find('h3', attrs={'class': 'usajobs-search-result__title'})
            name = name_box.text.strip().replace("\n","").replace(" ","")
            #title = job.find('h3',{'class':'usajobs-search-result__title'}).text.strip().replace("\n","").replace(" ","")
            #Append to final list
            name_scrape_list.append((name))
            
            #Get each agency
            #agency = job.find('h5',{'class':'usajobs-search-result__agency'}).text.strip().replace("\n","").replace(" ","")
            #Append to final list
            #agency_scrape_list.append((agency))
        
        #Flatten to one list without duplicates
        id_list = [val for sublist in id_scrape_list for val in sublist]
        
        #name_list = [val for sublist in name_scrape_list for val in sublist]
        
        #agency_list = [val for sublist in agency_scrape_list for val in sublist]
        
        #Create data frame
        #a = {'ID': id_list, 'Name': name_list}
        #df = pd.DataFrame(data=a)
        #df.to_json("job_id.JSON")
        
    #print(name)
    print(name_scrape_list)
    print(len(name_scrape_list))
    #print(df)
    print(id_list)
    print(len(id_list))
    #print(agency_scrape_list)
    #print(len(agency_scrape_list))
    
scrape()

['Actuary-DH', 'Actuary-DH', 'ACTUARY', 'PublicNoticeforMathematicianPositions', 'PublicNoticeforMathematicianPositions', 'ResearchEconomistResearchMathematicianResearchStatistician', 'ResearchEconomistResearchMathematicianResearchStatistician', 'ResearchEconomistResearchMathematicianResearchStatistician', 'ResearchInterdisciplinary', 'INTERDISCIPLINARYENGINEER/SCIENTIST', 'Mathematician(ProgramDirector)', 'ENGINEER/SCIENTIST', 'InterdisciplinaryEngineer/Scientist', 'ResearchInterdisciplinary', 'SCIENCEADVISOR(INTERDISCIPLINARY)', 'Interdisciplinary(HeadoftheTechnologyTransferOffice)', 'TECHNICALPROJECTMANAGER', 'Interdisciplinary', 'INTERDISCIPLINARYENGINEER/SCIENTIST', 'TECHNICALPROJECTMANAGER', 'Interdisciplinary', 'ENGINEER/SCIENTIST', 'MANAGER', 'INTERDISCIPLINARYENGINEER/SCIENTIST', 'ENGINEER/SCIENTIST', 'INTERDISCIPLINARY', 'MathematicalStatistician,GS-1529-7/9,Census-DE-HLF', 'MathematicalStatistician,GS-1529-11/12,Census-DHA-HLF', 'MathematicalStatistician(ProgramDirector)', '

In [5]:
#data frame validation 
from bs4 import BeautifulSoup
from helium import *
import time 
from datetime import date
import pandas as pd 
import json

def scrape1():
    url = 'https://www.usajobs.gov/?c=opportunities'
    driver = start_chrome(url, headless=True)
    driver.get(url)
                   
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    hrefs_list = []
    jobs_list = []
    scrape_list = []
    
    
    #for link in soup.find_all("a",{"class": "usajobs-landing-find-opportunities__job"}):
       # hrefs_list.append(link['href'])
    for p in soup.find_all("p",{"class": "usajobs-landing-find-opportunities__desc-left"}):
        for special_link in p.find_all("a"):
            hrefs_list.append(special_link['href'])
    for term in hrefs_list:
        new_url = 'https://www.usajobs.gov'+term
        driver = start_chrome(new_url, headless=True)
        driver.get(new_url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")       
        job_link = soup.find_all("a",{'class':'usajobs-search-result--card__body'})       
        titles_list = []
        agencies_list = []
        summaries_list = []
        application_times_list = []
        c = True
        while c == True:
            try:
                for job in job_link:
                    application_time = job.find('p',{'class':'usajobs-search-result__closing-date'}).text.strip().replace("\n","").replace("Closing date Closes ","")
                    today = date.today().strftime("%m/%d%y")
                    if today == application_time:
                        break
                    title = job.find('h3',{'class':'usajobs-search-result__title'}).text.strip().replace("\n","").replace(" ","")
                    agency = job.find('h5',{'class':'usajobs-search-result__agency'}).text.strip().replace("\n","")
                    summary = job.find('p',{'class':'usajobs-search-result__multi-line'}).text.strip().replace("\n","").replace("•","").replace(" ","")
                    titles_list.append(title)
                    agencies_list.append(agency)
                    application_times_list.append(application_time)
                    summaries_list.append(summary)
                    scrape_list.append((title,application_time,agency,summary)) 
                driver.find_element_by_class_name('usajobs-search-pagination__next-page').click()
                time.sleep(1)
            except:
                c = False
    a = {'Job name': titles_list,'agencies':agencies_list}
    b = {'Job name': titles_list,'Time close':application_times_list}
    c= {'Job name': titles_list,'Summary':summaries_list}
    d = {'Job name': titles_list,'agencies':agencies_list, 'Time close':application_times_list,'Summary':summaries_list}
    data_frame1 = pd.DataFrame(data=a)
    data_frame2 = pd.DataFrame(data=b)
    data_frame3 = pd.DataFrame(data=c)
    data_frame4 = pd.DataFrame(data=d)
    data_frame1.to_csv( "agencies_list.csv")
    data_frame2.to_csv( "application_time_list.csv")
    data_frame3.to_csv( "summaries_list.csv")
    data_frame4.to_json( "job_list.JSON")
    
    
    print(data_frame1)
    print(data_frame2)
    print(data_frame3)
 
    
scrape1()


                                 Job name               agencies
0                             Optometrist  Indian Health Service
1                             Optometrist  Indian Health Service
2             SuicidePreventionSpecialist  Indian Health Service
3          MedicalRecordTechnician(Coder)  Indian Health Service
4           ChildandYouthProgramAssistant   Air Mobility Command
..                                    ...                    ...
320            HealthTechnician(Cultural)  Indian Health Service
321  LeadDiagnosticRadiologicTechnologist  Indian Health Service
322  LeadDiagnosticRadiologicTechnologist  Indian Health Service
323                    PharmacyTechnician  Indian Health Service
324               PharmacyTechnician(POS)  Indian Health Service

[325 rows x 2 columns]
